In [1]:
import os
%pwd

'c:\\Users\\Raghavan\\Documents\\CropsAndWeedsSegmentation\\research'

In [2]:
os.chdir('../')

In [11]:
## config 
from PIL import Image

img = Image.open('artifacts/data_ingestion/train/img/20824007_frame_000015.jpg')
mask = Image.open('artifacts/data_ingestion/train/mask/20824007_frame_000015.png')

img.size == (224,2724)

False

In [ ]:
from PIL.JpegImagePlugin import JpegImageFile
from PIL.PngImagePlugin import PngImageFile
isinstance(img,PngImageFile)

True

In [15]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataValidationConfig:
    root_dir: Path
    data_dir: Path
    STATUS_FILE: Path
    all_schema: dict




In [16]:
from src.cropsAndWeedsSegmentation.constants import *
from src.cropsAndWeedsSegmentation.utils.common import read_yaml,create_directories

class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,params_filepath = PARAMS_FILE_PATH,schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self,mongo_uri:str)->DataValidationConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir,config.train_img_dir,config.train_mask_dir,config.test_img_dir,config.test_mask_dir,config.val_img_dir,config.val_mask_dir,config.others_img_dir])  

        data_ingestion_config = DataValidationConfig(
            root_dir=config.root_dir,
            
            train_img_dir=config.train_img_dir,
            train_mask_dir= config.train_mask_dir,

            test_img_dir=config.test_img_dir,
            test_mask_dir= config.test_mask_dir,

            val_img_dir=config.val_img_dir,
            val_mask_dir= config.val_mask_dir,

            others_img_dir=config.others_img_dir,

            mongo_uri=mongo_uri,
            database_name=config.database_name,
            collection_name=config.collection_name
        )      
        return data_ingestion_config
    
    def get_data_validation_config(self)->DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.InputImg

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            data_dir=config.data_dir,
            all_schema=schema
        )

        return data_validation_config
    
    



In [33]:
from src.cropsAndWeedsSegmentation.logging.logger import logger
import os
from PIL import Image
from PIL.JpegImagePlugin import JpegImageFile

class DataValidation:
    def __init__(self,config:DataValidationConfig):
        self.config = config
        self.type = {
            "JpegImageFile": JpegImageFile
        }

    def validate_dataset(self)->bool:
        validation_status = False
        folders = ['train','test','val','others']
        for folder in folders:
            folder_path = os.path.join(self.config.data_dir,folder,'img')
            for img_file in os.listdir(folder_path):
                img_filePath = os.path.join(folder_path,img_file)
                img = Image.open(img_filePath)

                if not isinstance(img,self.type[self.config.all_schema.type]) and img.size != (self.config.all_schema.height,self.config.all_schema.width):
                    with open(self.config.STATUS_FILE,'w') as file:
                        file.write(f'Validation status: False')
                    return False
                
        with open(self.config.STATUS_FILE,'w') as file:
            file.write(f'Validation status: True')
        return True


In [34]:
try:
    config = ConfigurationManager()
    data_validaion_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validaion_config)
    data_validation.validate_dataset()
except Exception as e:
    raise e

[2025-03-09 22:29:49,851: INFO: common: Yaml file: config\config.yaml loaded successfully]
[2025-03-09 22:29:49,854: INFO: common: Yaml file: params.yaml loaded successfully]
[2025-03-09 22:29:49,859: INFO: common: Yaml file: schema.yaml loaded successfully]
[2025-03-09 22:29:49,865: INFO: common: Created directory at: artifacts]
[2025-03-09 22:29:49,867: INFO: common: Created directory at: artifacts/data_validation]
